In [1]:
import sys
sys.path.append("../")
import gpt
import logging as log
log.getLogger().setLevel(log.INFO)

In [14]:
file ="/home/luca/Documenti/sezione_copernicus/tmp/PM-MOO-MS-Copernicus_matteo.gpkg"
file= "/home/luca/Downloads/PM-MAR-MS-Arsinoes_01.gpkg"
gp = gpt.read_file(file)

INFO: Failed to auto identify EPSG: 7
INFO: Failed to auto identify EPSG: 7
INFO: Failed to auto identify EPSG: 7
INFO: Failed to auto identify EPSG: 7


In [15]:
gp.list()

['Structural features',
 'Surface features',
 'Units',
 'Units_contacts',
 'layer_styles']

In [21]:
layer_rename = {"Units": "geounits", "Structural features": "structural"}


def rename_layers(gpk, rename_patterns):
    for oldname, newname in rename_patterns.items():
        # do rename and log it for future inspection
        if oldname in gpk.list():
            log.info(f"renaming layer {oldname} to {newname}")
            gpk.rename_layer(oldname, newname)
        else:
            log.info(f"layer {oldname} not found. Nothing done")
    
rename_layers(gp, layer_rename)

for l in gp.list():
    table = gp[l]
    print(f"{l} : {list(table.columns)}" )

INFO: layer Units not found. Nothing done
INFO: layer Structural features not found. Nothing done




Surface features : ['Id', 'Unit', 'geometry']
Units_contacts : ['Unit', 'Z_Min', 'Z_Max', 'Z_Mean', 'Length3D', 'Min_Slope', 'Max_Slope', 'Avg_Slope', 'Vertex_Cnt', 'geometry']
geounits : ['Unit', 'Z_Min', 'Z_Max', 'Z_Mean', 'Length3D', 'Min_Slope', 'Max_Slope', 'Avg_Slope', 'Vertex_Cnt', 'geometry']
layer_styles : ['f_table_catalog', 'f_table_schema', 'f_table_name', 'f_geometry_column', 'styleName', 'styleQML', 'styleSLD', 'useAsDefault', 'description', 'owner', 'ui', 'update_time', 'geometry']
structural : ['Unit', 'Z_Min', 'Z_Max', 'Z_Mean', 'Length3D', 'Min_Slope', 'Max_Slope', 'Avg_Slope', 'Vertex_Cnt', 'Id', 'layer', 'path', 'Type', 'geometry']


In [22]:
field_rename =  {".*": {"Unit": "geo_type"}, "geounits":{"Z_Min":"zmin"}} # {regex for layer_name (.* for all) : {from:to}}

import re
def rename_fields(gpk, rename_patterns):
    layers =gpk.list()
    for layer_matcher, pattern in rename_patterns.items():
        for layer in layers:
            if re.match(layer_matcher, layer):
                # do the rename if the field exists
                for fromname, toname in pattern.items():
                    if fromname in list(gpk[layer].columns):
                        log.info(f"renaming field {fromname} of layer {layer} to {toname}")
                        gpk.rename_field(layer, fromname, toname)
                    else:
                        log.info(f"Layer {layer} match but field not present. Nothing to do")
        
        
rename_fields(gp, field_rename)   

INFO: renaming field Unit of layer Surface features to geo_type
INFO: updating filter def Unit = 'Pitted area' to geo_type = 'Pitted area'
INFO: updating filter def Unit = 'Pit chain' to geo_type = 'Pit chain'
INFO: renaming field Unit of layer Units_contacts to geo_type
INFO: updating filter def Unit = 'Chaotic Terrain - Knobby terrains'  to geo_type = 'Chaotic Terrain - Knobby terrains' 
INFO: updating filter def Unit = 'Cap Unit'  to geo_type = 'Cap Unit' 
INFO: updating filter def Unit = 'Chaotic Terrain - High Thermal Inertia'  to geo_type = 'Chaotic Terrain - High Thermal Inertia' 
INFO: updating filter def Unit = 'Light-toned Layered Unit'  to geo_type = 'Light-toned Layered Unit' 
INFO: renaming field Unit of layer geounits to geo_type
INFO: updating filter def Unit = 'Post-collapse craters' to geo_type = 'Post-collapse craters'
INFO: updating filter def Unit = 'Cap Unit' to geo_type = 'Cap Unit'
INFO: updating filter def Unit = 'Light-toned Layered Unit' to geo_type = 'Light-t

In [23]:
for l in gp.list():
    table = gp[l]
    print(f"{l} : {list(table.columns)}" )



Surface features : ['Id', 'geo_type', 'geometry']
Units_contacts : ['geo_type', 'Z_Min', 'Z_Max', 'Z_Mean', 'Length3D', 'Min_Slope', 'Max_Slope', 'Avg_Slope', 'Vertex_Cnt', 'geometry']
geounits : ['geo_type', 'zmin', 'Z_Max', 'Z_Mean', 'Length3D', 'Min_Slope', 'Max_Slope', 'Avg_Slope', 'Vertex_Cnt', 'geometry']
layer_styles : ['f_table_catalog', 'f_table_schema', 'f_table_name', 'f_geometry_column', 'styleName', 'styleQML', 'styleSLD', 'useAsDefault', 'description', 'owner', 'ui', 'update_time', 'geometry']
structural : ['geo_type', 'Z_Min', 'Z_Max', 'Z_Mean', 'Length3D', 'Min_Slope', 'Max_Slope', 'Avg_Slope', 'Vertex_Cnt', 'Id', 'layer', 'path', 'Type', 'geometry']
